Import module printing the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#READ DATA
data = pd.read_csv("C:\\Users\\USER\\Desktop\\New Research\\dataset.csv")
data.head(10)

,label,Text
0,1,@polythenespam I will be all packed for rehab....
1,1,Sorry if you at having trouble viewing my stor...
2,1,Ah! My pc is so bad to me!!!
3,1,"@kstew_fan No shade, still all old! And you h..."
4,1,Our phone line is dead.
5,1,Spending my last days in Washington with the b...
6,0,http://twitpic.com/6ivhj - Demi...Ã¢ÂÂ¥Ã¢ÂÂ¥...
7,1,"@missxseptemberk nope, because then I'd miss m..."
8,0,@JillzWorth its sooooooooo gone i feel so a...
9,1,lost ma phone... man this is da worst bday wee...


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
import nltk
from nltk.corpus import stopwords
import string
import re

In [3]:
# Sample data

sample_data = data.sample(150000,replace=True, random_state=1)
X = sample_data['Text']
y = sample_data['label']
print(X)
print(y)

128037    @littleangel93 I think i'm falling in love wit...
5192      looking for recipes for diabetics, that actual...
50057                                 @danbimrose anytime! 
109259    Todays shift at work is 10:30 - 7:15  I called...
73349                       Where are all ma girls tonight 
                                ...                        
31132     hix,.... good bye my friends, good bye School ...
42761     @tracieshu its super cute. we watched it after...
120772    @courtneyyy3 i know. you're missing late night...
49752     @tommcfly hey tom  say hi to me, please please...
48941     What does a man have to do to get a freaking g...
Name: Text, Length: 150000, dtype: object
128037    0
5192      1
50057     0
109259    1
73349     1
         ..
31132     1
42761     1
120772    1
49752     1
48941     1
Name: label, Length: 150000, dtype: int64


# STOP WORDS

In [4]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# CLEAN DATA

In [5]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import re
import string
import nltk
import spacy
import pandas as pd
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from spacy.lang.en.stop_words import STOP_WORDS as spacy_stopwords
from collections import Counter

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Stopwords set (combining NLTK and spaCy stopwords)
stop_words = frozenset(stopwords.words('english')).union(spacy_stopwords)

# Dictionary of contractions and their expanded forms
abbreviation_map = {
    "isn't": "is not", "aren't": "are not", "can't": "cannot",
    "couldn't": "could not", "don't": "do not", "doesn't": "does not",
    "n't": "not", "'re": "are", "'ve": "have", "'ll": "will",
    "'d": "would", "'s": "is"
}

# Enhanced data cleaning function
def clean_text(text):
    # Convert to lowercase and handle abbreviations
    text = text.lower()
    for short_form, full_form in abbreviation_map.items():
        text = re.sub(rf"\b{re.escape(short_form)}\b", full_form, text)
    
    # Combine all regex replacements into a single call
    text = re.sub(
        r'http\S+|www\S+|https\S+|'        # URLs
        r'\b\w+@\w+\.\w+\b|'               # Emails
        r'@\w+|#\w+|'                      # Hashtags and mentions
        r'\d+|'                            # Digits
        r'[^\x00-\x7F]+|'                  # Non-English symbols
        r'\s+', ' ',                       # Excessive whitespace
        text
    )
    
    # Remove punctuation and trim whitespace
    return text.translate(str.maketrans('', '', string.punctuation)).strip()

# Tokenize, lemmatize, and filter by POS and stopwords in one step
def process_and_filter_text(text):
    tokens = word_tokenize(text)
    pos_tagged = pos_tag(tokens)
    filtered_words = []
    
    for word, tag in pos_tagged:
        # Exclude stopwords and unwanted POS tags (e.g., prepositions and conjunctions)
        if word in stop_words or tag in {'IN', 'CC'}:
            continue
        
        # Determine POS and lemmatize accordingly
        if tag.startswith('NN'):
            lemma = lemmatizer.lemmatize(word, pos='n')
        elif tag.startswith('VB'):
            lemma = lemmatizer.lemmatize(word, pos='v')
        elif tag.startswith('JJ'):
            lemma = lemmatizer.lemmatize(word, pos='a')
        elif tag.startswith('RB'):
            lemma = lemmatizer.lemmatize(word, pos='r')
        else:
            lemma = lemmatizer.lemmatize(word)
        
        filtered_words.append((lemma, tag))  # Append word with POS tag if it passes all filters
    
    return filtered_words

# Full data cleaning and preprocessing pipeline
def preprocess_data(text):
    cleaned_text = clean_text(text)
    return process_and_filter_text(cleaned_text)

# Apply the preprocessing function to the dataset
data = pd.read_csv("C:\\Users\\USER\\Desktop\\New Research\\dataset.csv")
data['Cleaned_Text_POS'] = data['Text'].apply(preprocess_data)

# Filter negative samples and analyze POS tags
negative_data = data[data['label'] == 1]

# Collect POS tags of words in negative texts
pos_tags_negative = Counter()
for row in negative_data['Cleaned_Text_POS']:
    pos_tags_negative.update(tag for _, tag in row)

# Display the most common POS tags associated with negative words
print("Most common POS tags in negative words:")
print(pos_tags_negative.most_common(10))

# Remove the POS tags to retain only the cleaned text for other uses
data['Cleaned_Text'] = data['Cleaned_Text_POS'].apply(lambda x: ' '.join(word for word, _ in x))

# Display the first 20 rows of the cleaned dataset with POS analysis
print(data.head(20))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Most common POS tags in negative words:
[('NN', 279060), ('JJ', 98524), ('VB', 51556), ('NNS', 50896), ('VBP', 39925), ('VBG', 37556), ('VBD', 28735), ('RB', 26769), ('VBN', 19943), ('VBZ', 13296)]
    label                                               Text  \
0       1  @polythenespam I will be all packed for rehab....   
1       1  Sorry if you at having trouble viewing my stor...   
2       1                      Ah! My pc is so bad to me!!!    
3       1  @kstew_fan No shade, still all old!  And you h...   
4       1                           Our phone line is dead.    
5       1  Spending my last days in Washington with the b...   
6       0  http://twitpic.com/6ivhj - Demi...Ã¢ÂÂ¥Ã¢ÂÂ¥...   
7       1  @missxseptemberk nope, because then I'd miss m...   
8       0  @JillzWorth its sooooooooo gone    i feel so a...   
9       1  lost ma phone... man this is da worst bday wee...   
10      0  @revjesse @TipSquirrel Haha... if I ruled the ...   
11      1  Not been tweeting much 

In [7]:

data.head()
data.shape
print(data.shape)
print(data.head())
print(data.columns)
print(X.head())
print(y.head())


(149900, 4)
   label                                               Text  \
0      1  @polythenespam I will be all packed for rehab....   
1      1  Sorry if you at having trouble viewing my stor...   
2      1                      Ah! My pc is so bad to me!!!    
3      1  @kstew_fan No shade, still all old!  And you h...   
4      1                           Our phone line is dead.    

                                    Cleaned_Text_POS  \
0  [(pack, VBN), (rehab, NN), (tell, VB), (date, ...   
1  [(sorry, NN), (trouble, NN), (view, VBG), (sto...   
2                    [(ah, VB), (pc, NN), (bad, JJ)]   
3  [(shade, NN), (old, JJ), (new, JJ), (pesquisar...   
4              [(phone, NN), (line, NN), (dead, JJ)]   

                                   Cleaned_Text  
0  pack rehab tell date bruno twittersober lmao  
1      sorry trouble view story site tech issue  
2                                     ah pc bad  
3                       shade old new pesquisar  
4                     

# DISTILBERT

In [8]:
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Optional: Disable W&B logging
import os
os.environ["WANDB_DISABLED"] = "true"

# Load your cleaned dataset
data = pd.read_csv("C:\\Users\\USER\\Desktop\\New Research\\dataset.csv")

# Strip whitespace from column names
data.columns = data.columns.str.strip()

# Ensure 'Text' and 'label' columns exist
if 'Text' not in data.columns or 'label' not in data.columns:
    raise KeyError("Columns 'Text' or 'label' not found in the dataset.")

# Fill NaN values and ensure all entries are strings
data['Text'] = data['Text'].fillna('').astype(str)

# Sample the data if it has more than 30,000 entries
if len(data) > 30000:
    data = data.sample(n=30000, random_state=42)

# Prepare the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', 
    num_labels=len(data['label'].unique())
)

# Prepare the input texts and labels
texts = data['Text'].tolist()
labels = data['label'].astype('category').cat.codes.tolist()

# Split data into training and evaluation sets
train_texts, eval_texts, train_labels, eval_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=128)

# Create a custom torch dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
eval_dataset = CustomDataset(eval_encodings, eval_labels)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./distilbert_results',
    run_name='my_distilbert_experiment',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Define a compute_metrics function to calculate accuracy and other metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define Trainer with compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics  # Include compute_metrics here
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("distilbert_model")

# Evaluate the model
eval_metrics = trainer.evaluate()
if 'eval_accuracy' in eval_metrics:
    print(f"DistilBERT Evaluation Accuracy: {eval_metrics['eval_accuracy']:.2f}")
else:
    print("Accuracy not found in evaluation metrics.")


C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\USER\AppData\Roaming\Python\Python310\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
  2%|▏         | 100/4500 [12:04<12:52:22, 10.53s/it]

{'loss': 0.522, 'grad_norm': 4.671262741088867, 'learning_rate': 4.888888888888889e-05, 'epoch': 0.07}


  4%|▍         | 200/4500 [21:05<2:26:36,  2.05s/it] 

{'loss': 0.4661, 'grad_norm': 5.349675178527832, 'learning_rate': 4.7777777777777784e-05, 'epoch': 0.13}


...

{'eval_loss': 0.6782447099685669, 'eval_accuracy': 0.8411666666666666, 'eval_f1': 0.838793386726796, 'eval_precision': 0.8383921383168066, 'eval_recall': 0.8411666666666666, 'eval_runtime': 115.1127, 'eval_samples_per_second': 52.123, 'eval_steps_per_second': 6.515, 'epoch': 3.0}


100%|██████████| 4500/4500 [2:51:18<00:00,  2.28s/it]


{'train_runtime': 10278.4266, 'train_samples_per_second': 7.005, 'train_steps_per_second': 0.438, 'train_loss': 0.2697330820295546, 'epoch': 3.0}


100%|██████████| 750/750 [01:54<00:00,  6.57it/s]

DistilBERT Evaluation Accuracy: 0.84
